<a href="https://colab.research.google.com/github/SydanJainen/Cell-image-segmentation/blob/main/image_Segmantation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HYPERPARAMETER

In [1]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNEL = 3

BATCH_SIZE = 32
EPOCHS = 3
PADDING = 1
STRIDE = 1

VAL_SPLIT = 0.2
SEED = 23

LEARNING_RATE = 1e-4

CHECK IF COLAB OR OFFLINE

In [2]:
if 'google.colab' in str(get_ipython()):
  isColab = True
else:
  isColab = False

CHECK IF GPU IS AVAILABLE

In [3]:
import torch

num_gpu = torch.cuda.device_count()
print(f'{num_gpu} GPU available')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

1 GPU available


REQUIREMENT

In [4]:
if isColab:
  print("Check if drive is mounted, chek if mask is label created")
  from google.colab import drive
  drive.mount('/content/drive')
  DB_PATH = '/content/drive/MyDrive/scuola/deeplife/'
else:
  print("Check if folder is correct")
  DB_PATH = 'database/'

Check if drive is mounted, chek if mask is label created
Mounted at /content/drive


IMPORTS

In [5]:
import os
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A

import torch.nn as nn
import torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

LAOD DATA

In [6]:
IMAGE_PATH = os.path.join(DB_PATH, "train")
LABEL_PATH = os.path.join(DB_PATH, "label")

# Ensure both directories have the same number of files
number_images = len(os.listdir(IMAGE_PATH))
number_labels = len(os.listdir(LABEL_PATH))

assert number_images == number_labels

# LOAD PIL IMAGES AND LABELS
image_files = sorted(os.listdir(IMAGE_PATH))
label_files = sorted(os.listdir(LABEL_PATH))

# Split into train and temp sets
train_images, val_images, train_labels, val_labels = train_test_split(image_files, label_files, test_size=VAL_SPLIT, random_state=SEED)


DEFINE DATASET

In [19]:
class CellularDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        """
        Args:
            image_dir (string): Directory with all the images.
            label_dir (string): Directory with all the labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.image_files = images
        self.label_files = labels

        # Ensure both directories have the same number of files
        assert len(self.image_files) == len(self.label_files), \
            "The number of images and labels should be the same"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Construct file paths
        img_name = os.path.join(IMAGE_PATH, self.image_files[idx])
        label_name = os.path.join(LABEL_PATH, self.label_files[idx])

        # Load image and label
        image = Image.open(img_name).convert('RGB')
        label = Image.open(label_name).convert('L')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)
            label = self.transform(label)

        # Convert to NumPy arrays
        image_np = np.array(image)
        label_np = np.array(label)

        # Convert to PyTorch tensors
        image_tensor = torch.from_numpy(image_np).float()
        label_tensor = torch.from_numpy(label_np).long()

        return image_tensor, label_tensor

DEFINE TRANSFORMATIONS

Create Dataset and Dataloader

In [20]:
train_dataset = CellularDataset(train_images,train_labels)
val_dataset = CellularDataset(val_images,val_labels)

In [21]:
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)


CHECK IF TRAIN DATASET IS LOADED CORRECTLY

In [10]:
'''
images, labels = next(iter(train_loader))

idx = random.randint(0, images.size(0) - 1)

# Extract the image and label
image = images[idx]
label = labels[idx]

# Convert the image tensor to a numpy array
image_np = image.numpy()

# Plot the image
plt.figure(figsize=(5, 5))
plt.imshow(np.transpose(image_np, (1, 2, 0)))
if label.dim() == 0:
  # Label is a single value (class label)
  plt.title(f'Label: {label.item()}')
else:
  # Label is an image
  label_np = label.numpy()
  plt.figure(figsize=(5, 5))
  plt.imshow(np.transpose(label_np, (1, 2, 0)), cmap='gray')
  plt.title('Label Image')
plt.axis('off')
plt.show()
'''

"\nimages, labels = next(iter(train_loader))\n\nidx = random.randint(0, images.size(0) - 1)\n\n# Extract the image and label\nimage = images[idx]\nlabel = labels[idx]\n\n# Convert the image tensor to a numpy array\nimage_np = image.numpy()\n\n# Plot the image\nplt.figure(figsize=(5, 5))\nplt.imshow(np.transpose(image_np, (1, 2, 0)))\nif label.dim() == 0:\n  # Label is a single value (class label)\n  plt.title(f'Label: {label.item()}')\nelse:\n  # Label is an image\n  label_np = label.numpy()\n  plt.figure(figsize=(5, 5))\n  plt.imshow(np.transpose(label_np, (1, 2, 0)), cmap='gray')\n  plt.title('Label Image')\nplt.axis('off')\nplt.show()\n"

MODEL

In [22]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

TRAIN

In [12]:
if isColab:
  !pip install tqdm

In [23]:
from tqdm import tqdm

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.permute(0, 3, 1, 2).to(device)  # Transpose to [batch_size, channels, height, width]
        targets = targets.float().unsqueeze(1).to(device)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.permute(0, 3, 1, 2).to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def fit(model, optimizer, loss_fn, scaler):
  for epoch in range(EPOCHS):
    train_fn(train_loader, model, optimizer, loss_fn, scaler)

    # check accuracy
    check_accuracy(val_loader, model, device=device)

In [24]:
model = UNET(in_channels=3, out_channels=1).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

In [25]:
fit(model,optimizer,loss_fn,scaler)

  0%|          | 0/17 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 17/17 [04:07<00:00, 14.59s/it, loss=-78.1]


Got 7288149/8781824 with acc 82.99
Dice score: 1.0524696111679077


100%|██████████| 17/17 [00:15<00:00,  1.07it/s, loss=-140]


Got 7110158/8781824 with acc 80.96
Dice score: 1.8139652013778687


100%|██████████| 17/17 [00:15<00:00,  1.10it/s, loss=-170]


Got 7162560/8781824 with acc 81.56
Dice score: 1.903640627861023
